In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('https://bit.ly/kaggletrain')

In [6]:
df.shape

(891, 12)

In [8]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [9]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [12]:
df = df.loc[df.Embarked.notna(), ["Survived", "Pclass", "Sex", "Embarked"]]

In [14]:
df.shape

(889, 4)

In [15]:
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In [33]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd


class GroupMedianImputer(BaseEstimator, TransformerMixin):
    def __init__(self, group_col, target_col):
        self.group_col = group_col
        self.target_col = target_col

    def fit(self, X, y=None):
        df = pd.DataFrame(X)
        self.group_medians_ = (
            df.groupby(self.group_col)[self.target_col].median()
        )
        self.global_median_ = df[self.target_col].median()
        return self

    def transform(self, X):
        df = pd.DataFrame(X).copy()
        df[self.target_col] = df[self.target_col].fillna(
            df[self.group_col].map(self.group_medians_)
        )
        df[self.target_col] = df[self.target_col].fillna(self.global_median_)
        return df

In [37]:
from sklearn.impute import SimpleImputer
import pandas as pd

df = pd.DataFrame({"v1": [1, 2, 3, None], "v2": ["A", "B", "A", "B"]})
df2 = pd.DataFrame({"v1": [1, 2, 3, 4, 2, None], "v2": [
                   "A", "B", "B", "A", "B", "C"]})

si = SimpleImputer()
wgmi = GroupMedianImputer("v2", "v1")

wgmi.fit_transform(df)
wgmi.transform(df2)

,v1,v2
0,1.0,A
1,2.0,B
2,3.0,B
3,4.0,A
4,2.0,B
5,2.0,C
